In [13]:
import sys # for automation and parallelization: set manual to false when run by a launcher
import json
default = {'scenario': 'port-moresby','training_folder':'../..'} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))

In [14]:
import os
import time
import geopandas as gpd
import pandas as pd
import numpy as np
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

num cores: 8


In [15]:
scenario = argv['scenario']


on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)

training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'

if not on_lambda:
    scenario_folder = training_folder + '/scenarios/' + scenario + '/inputs/'
    output_folder = training_folder + '/scenarios/' + scenario + '/outputs/'
    model_folder = training_folder + '/scenarios/' + scenario + '/model/'
else:
    scenario_folder = input_folder
    output_folder = training_folder + '/outputs/'
    model_folder = training_folder + '/model/'
print('input folder: ', input_folder)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folder)
print('model folder : ', model_folder)

On Lambda :  False
input folder:  ../../inputs/
output folder:  ../../scenarios/port-moresby/outputs/
scen folder :  ../../scenarios/port-moresby/inputs/
model folder :  ../../scenarios/port-moresby/model/


In [16]:
if 'params' in argv.keys():
    scenario = argv['params']['general']['scenario'] #'params':{'general':{'scenario':'ref2023'}}
    var = excel.read_var(file=input_folder+'parameters.xlsx', scenario=scenario, return_ancestry=False)
    var.update(pd.DataFrame.from_dict(argv['params'], orient="index").stack())
else:
    var = excel.read_var(file=input_folder+'parameters.xlsx', scenario=scenario, return_ancestry=False)

# inputs

In [17]:
sm = stepmodel.read_zippedpickles(model_folder + 'network')
distribution = stepmodel.read_zippedpickles(model_folder + 'distribution')

sm.volumes = distribution.volumes

zones_before_clustering: 100%|██████████| 2/2 [00:00<00:00, 1152.91it/s]


# od_set

In [18]:
v = sm.volumes.set_index(['origin', 'destination']).sum(axis=1)
od_set = set(v.loc[v>0].index)

In [19]:
cutoff = var["pathfinder"]["cutoff"]

# PT pathfinder

In [20]:

sm.step_pt_pathfinder(
    broken_routes=True,
    broken_modes=True,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    walk_on_road=True,
    path_analysis=False,
    od_set=od_set,
    cutoff=cutoff,
    verbose=False,
)
ppf = sm.publicpathfinder

start publicpathfinder


route_type {'PMV'}: 100%|██████████| 3/3 [00:00<00:00,  8.21it/s]       

path_analysis


In [21]:
sm.step_road_pathfinder(method='aon', access_time='time',volume_column= 'car_owner',  time_column='time', od_set=od_set,path_analysis=True )

In [22]:
sm.analysis_pt_los(walk_on_road=True)
sm.analysis_pt_route_type(hierarchy=[ 'car','rail', 'subway', 'tram', 'bus', 'PMV', 'NCD','walk'])

path_analysis: 100%|██████████| 7443/7443 [00:00<00:00, 31336.56it/s]


In [23]:
sm.to_zippedpickles(model_folder +'los', omitted_attributes=['publicpathfinder'])

car_los: 100%|██████████| 29/29 [00:00<00:00, 43.78it/s]      


In [24]:
end_of_notebook

NameError: name 'end_of_notebook' is not defined